In [1]:
# Clone the SleepQA repo
!git clone https://github.com/IvaBojic/SleepQA.git
%cd SleepQA

# Install key dependencies
!pip install transformers faiss-cpu datasets scikit-learn pandas tqdm


Cloning into 'SleepQA'...
remote: Enumerating objects: 400, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 400 (delta 46), reused 43 (delta 19), pack-reused 313 (from 1)
Receiving objects: 100% (400/400), 31.13 MiB | 7.20 MiB/s, done.
Resolving deltas: 100% (176/176), done.
Updating files: 100% (134/134), done.
Filtering content: 100% (3/3), 1.21 GiB | 4.80 MiB/s, done.
/content/SleepQA
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installa

In [6]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
import faiss
import numpy as np
import time
import torch
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt  # Add at the top if not already
from transformers import BertTokenizer, BertForQuestionAnswering

# 1. Load data
corpus = pd.read_csv("data/training/sleep-corpus.tsv", sep="\t", header=None)
corpus.columns = ['index', 'passage', 'title']
test_qs = pd.read_csv("data/training/sleep-test.csv", sep="\t", header=None)
test_qs.columns = ['question', 'answer']

# 2. Setup models
models = {
    "BERT-SQuAD2": "deepset/bert-base-cased-squad2",
    "BioBERT": "dmis-lab/biobert-base-cased-v1.1",
    "BioBERT-BioASQ": "ktrapeznikov/biobert_v1.1_pubmed_squad_v2",
    "PubMedBERT": "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract",
    "SciBERT": "allenai/scibert_scivocab_uncased",
    "ClinicalBERT": "emilyalsentzer/Bio_ClinicalBERT"
}

# 3. Prepare training examples
train_examples = [
    InputExample(texts=[row["question"], row["answer"]], label=1.0)
    for _, row in test_qs.iterrows()
]

# 4. Set hyperparameters
EPOCHS = 30
BATCH_SIZE = 16
LEARNING_RATE = 2e-5  # (cannot be directly set in SentenceTransformer.fit)

# Track compute
total_gpu_time = 0
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Hardware: Using {'GPU' if device == 'cuda' else 'CPU'}")

# 5. Define FAISS search
def query_faiss(question, model, top_k=3):
    q_emb = model.encode([question], convert_to_numpy=True)
    D, I = index.search(q_emb, top_k)
    return [answer_texts[i] for i in I[0]], D[0]

# 6. Define evaluation metrics
def evaluate_model(model, test_df, ks=[1]):
    results = {}

    for k in ks:
        recall_at_k = []
        exact_match = []
        f1_scores = []

        for _, row in test_df.iterrows():
            question = row["question"]
            true_answer = row["answer"]

            retrieved_passages, _ = query_faiss(question, model, top_k=k)

            # Recall@k: if true answer appears in any retrieved passage
            match_found = any(true_answer.strip().lower() in passage.lower() for passage in retrieved_passages)
            recall_at_k.append(1 if match_found else 0)

            # Exact Match
            em = any(true_answer.strip().lower() == passage.strip().lower() for passage in retrieved_passages)
            exact_match.append(1 if em else 0)

            # F1 score (token level) with best candidate
            best_f1 = 0
            for passage in retrieved_passages:
                true_tokens = set(true_answer.lower().split())
                pred_tokens = set(passage.lower().split())
                common = true_tokens & pred_tokens
                if not common:
                    continue
                precision = len(common) / len(pred_tokens)
                recall = len(common) / len(true_tokens)
                f1 = 2 * precision * recall / (precision + recall)
                best_f1 = max(best_f1, f1)
            f1_scores.append(best_f1)

        # Store results for each k
        results[k] = {
            "Recall@k": np.mean(recall_at_k),
            "Exact Match": np.mean(exact_match),
            "F1 Score": np.mean(f1_scores)
        }

    return results

# 7. Train and evaluate each model
for model_name, model_id in models.items():
    print(f"\n### Training Model: {model_name} ###")

    model = SentenceTransformer(model_id, device=device)
    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=BATCH_SIZE)
    train_loss = losses.CosineSimilarityLoss(model)

    start_time = time.time()
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=EPOCHS,
        warmup_steps=10,
        show_progress_bar=True
    )
    end_time = time.time()

    epoch_time = (end_time - start_time) / EPOCHS
    total_gpu_time += (end_time - start_time)

    print(f"Training time: {end_time - start_time:.2f}s total | {epoch_time:.2f}s per epoch")

    # Compute embeddings
    answer_texts = test_qs["answer"].tolist()
    answer_embeddings = model.encode(answer_texts, convert_to_numpy=True, show_progress_bar=True)
    index = faiss.IndexFlatL2(answer_embeddings.shape[1])
    index.add(answer_embeddings)


    # Show top-3 results for first question
    sample_question = test_qs.iloc[0]["question"]
    results, distances = query_faiss(sample_question, model, top_k=3)

    print("\nTop-3 Similar Passages:")
    for i, (text, dist) in enumerate(zip(results, distances)):
        print(f"Rank {i+1}: {text[:100]}... (Distance: {dist:.4f})")

    # Save model
    os.makedirs("saved_models", exist_ok=True)
    model_save_path = f"saved_models/{model_name.replace(' ', '_')}"
    model.save(model_save_path)
    print(f"Model saved to: {model_save_path}")

    # Evaluate model

    # Initialize metrics dictionary
    if 'all_metrics' not in locals():
        all_metrics = {}

    # Evaluate model
    metrics = evaluate_model(model, test_qs, ks=[1,20,40,60,80,100])
    all_metrics[model_name] = metrics

    print(f"\nEvaluation Metrics for {model_name}:")
    for k, metrics_at_k in metrics.items():
        print(f"- k={k}:")
        for metric_name, value in metrics_at_k.items():
            print(f"    {metric_name}: {value:.4f}")  # Format the value to 4 decimal places


# 8. Final report
print("\n=================== Training Summary ===================")
print(f"Hyperparameters Used:")
print(f"- Learning Rate: {LEARNING_RATE} (default for transformers)")
print(f"- Batch Size: {BATCH_SIZE}")
print(f"- Epochs: {EPOCHS}")
print(f"\nComputational Requirements:")
print(f"- Hardware: {'GPU' if device == 'cuda' else 'CPU'}")
print(f"- Models Trained: {len(models)}")
print(f"- Average Epoch Time (last model): {epoch_time:.2f}s")
print(f"- Total GPU Time: {total_gpu_time/3600:.2f} GPU hours")
print(f"\nTraining Details:")
print(f"- Loss Function: CosineSimilarityLoss")


Hardware: Using GPU

### Training Model: BERT-SQuAD2 ###


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

...

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss
500,0.000400


Training time: 185.16s total | 6.17s per epoch


Batches:   0%|          | 0/16 [00:00<?, ?it/s]


Top-3 Similar Passages:
Rank 1: ["the amount of time adults spent in different sleep stages"]... (Distance: 0.1247)
Rank 2: ["hormones that help control appetite and hunger"]... (Distance: 0.1307)
Rank 3: ["anyone who wants extra space for storage"]... (Distance: 0.1415)
Model saved to: saved_models/ClinicalBERT

Evaluation Metrics for ClinicalBERT:
- k=1:
    Recall@k: 0.0300
    Exact Match: 0.0300
    F1 Score: 0.0649
- k=20:
    Recall@k: 0.1460
    Exact Match: 0.1460
    F1 Score: 0.2813
- k=40:
    Recall@k: 0.2040
    Exact Match: 0.2040
    F1 Score: 0.3525
- k=60:
    Recall@k: 0.2420
    Exact Match: 0.2420
    F1 Score: 0.3892
- k=80:
    Recall@k: 0.2920
    Exact Match: 0.2920
    F1 Score: 0.4342
- k=100:
    Recall@k: 0.3400
    Exact Match: 0.3400
    F1 Score: 0.4820

=================== Training Summary ===================
Hyperparameters Used:
- Learning Rate: 2e-05 (default for transformers)
- Batch Size: 16
- Epochs: 30

Computational Requirements:
- Hardware: GP

In [7]:
# Evaluate model
#from sentence_transformers import SentenceTransformer

# Load the saved model from the specified path
#model = SentenceTransformer("saved_models/PubMedBERT")

# Now you can use the model for encoding, querying, etc.

#metrics = evaluate_model(model, test_qs, ks=[1,20,40,60,80,100])
#all_metrics[model_name] = metrics

#print(f"\nEvaluation Metrics for {model_name}:")
#print(f"\nEvaluation Metrics for {model_name}:")
#for k, metrics_at_k in metrics.items():
    #print(f"- k={k}:")
    #for metric_name, value in metrics_at_k.items():
        #print(f"    {metric_name}: {value:.4f}")  # Format the value to 4 decimal places
# 8. Final report
#print("\n=================== Training Summary ===================")
#print(f"Hyperparameters Used:")
#print(f"- Learning Rate: {LEARNING_RATE} (default for transformers)")
#print(f"- Batch Size: {BATCH_SIZE}")
#print(f"- Epochs: {EPOCHS}")
#print(f"\nComputational Requirements:")
#print(f"- Hardware: {'GPU' if device == 'cuda' else 'CPU'}")
#print(f"- Models Trained: {len(models)}")
#print(f"- Average Epoch Time (last model): {epoch_time:.2f}s")
#print(f"- Total GPU Time: {total_gpu_time/3600:.2f} GPU hours")
#print(f"\nTraining Details:")
#print(f"- Loss Function: CosineSimilarityLoss")

